In [1]:
##IMPORTING THE REQUIRED LIBRARIES

from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import pyspark.sql.functions as func
import time

## Function for Changing Data Type

In [2]:
##Function for chaging the datatype
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

## UDF for bucketing fields from HOTEL_DIM table

In [3]:
#star rating buckets
def srch_region_star_rating(star_rating):
    if star_rating == '2.0':
        return '2'
    elif star_rating == '2.5':
        return '2.5'
    elif star_rating == '3.0':
        return '3'
    elif star_rating == '3.5':
        return '3.5'
    elif star_rating == '4.0':
        return '4'
    elif star_rating == '1.0' or star_rating == '1.5':
        return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0':
        return '4.5-5'
    else:
        return '0'

In [4]:
#udf for hotel type
def srch_region_Hotel_type(Hotel_type,model_name):
	if model_name == 'France':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'RESIDENCE':
			return 'RESIDENCE'
		elif Hotel_type == 'APART-HOTEL' or Hotel_type == 'APARTMENT':
			return 'Family_Premium'
		else:
			return 'Others'
	elif model_name == 'TAM':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'HOTEL RESORT':
			return 'HOTEL RESORT'
		elif Hotel_type == 'RIAD':
			return 'RIAD'
		elif Hotel_type in ('GUEST HOUSE','HOSTEL/BACKPACKER ACCOMMODATION','BED & BREAKFAST','LODGE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL','ALL-INCLUSIVE','APARTMENT','RESIDENCE','VILLA','INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	elif model_name == 'Europe1':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'RESIDENCE':
			return 'RESIDENCE'
		elif Hotel_type == ('HOSTEL/BACKPACKER ACCOMMODATION','BED & BREAKFAST','GUEST HOUSE','HOSTAL (BUDGET HOTEL)','PENSION'):
			return 'Budget_hotel'
		elif Hotel_type in ('APARTMENT','APART-HOTEL','INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	elif model_name=="Europe2":
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == ('HOSTEL/BACKPACKER ACCOMMODATION','BED & BREAKFAST','GUEST HOUSE','HOSTAL (BUDGET HOTEL)','PENSION'):
			return 'Budget_hotel'
		elif Hotel_type == ('APARTMENT','APART-HOTEL','TOWNHOUSE','INN','RESIDENCE'):
			return 'Family_Premium'
		else:
			return 'Others'
	elif model_name == 'Other Region':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == ('HOSTEL/BACKPACKER ACCOMMODATION','GUEST HOUSE','BED & BREAKFAST'):
			return 'Budget_hotel'
		elif Hotel_type == ('MOTEL','LODGE','CABIN'):
			return 'Budget_hotel2'
		elif Hotel_type in ('HOTEL RESORT','INN','RYOKAN','POUSADA (BRAZIL)','CARAVAN PARK','PRIVATE VACATION HOME'):
			return 'Family_Premium'
		elif Hotel_type in ('APARTMENT','APART-HOTEL','ALL-INCLUSIVE','CONDOMINIUM RESORT','CONDO','VILLA','RESIDENCE'):
			return 'Family_Premium2'
		else:
			return 'Others'
	else:
		return 'NOT APPLICABLE'

In [5]:
##udf for market bucket
def srch_region_market(market,model_name):
	if model_name == 'France':
		if market == 'PARIS, FRANCE':
			return 'PARIS, FRANCE'
		elif market == 'LYON, FRANCE':
			return 'LYON, FRANCE'
		elif market == 'MONTPELLIER, FRANCE':
			return 'MONTPELLIER, FRANCE'
		elif market == 'MARSEILLE, FRANCE':
			return 'MARSEILLE, FRANCE'
		elif market in ('VAR/ESTEREL, FRANCE','SKI ALPES DU NORD, FRANCE','CANNES, FRANCE','RHONE ALPES, FRANCE','DEAUVILLE, FRANCE','BIARRITZ, FRANCE','NORMANDIE CITIES, FRANCE'):
			return 'G5'
		elif market in ('NICE, FRANCE','STRASBOURG, FRANCE','DISNEYLAND, FRANCE','ROUSSILLON, FRANCE','BORDEAUX, FRANCE','FRENCH RIVIERA, FRANCE','TOULOUSE, FRANCE','AQUITAINE, FRANCE','LILLE, FRANCE'):
			return 'G6'
		elif market in ('PARIS (AND VICINITY)','CDG-ROISSY-CHARLES DE GAULLE, PARIS, FRANCE','PAYS DE LA LOIRE-NANTES, FRANCE'):
			return 'G7'
		else:
			return 'others'
	
	elif model_name == 'TAM':
		if market == 'MARRAKECH, MOROCCO':
			return 'MARRAKECH, MOROCCO'
		elif market in ('PHUKET, THAILAND','KOH SAMUI, THAILAND','TEL AVIV, ISRAEL','ISRAEL','KOH PHANGAN, THAILAND','TANGIER, MOROCCO','MOROCCO','ESSAOUIRA, MOROCCO','FES, MOROCCO'):
			return 'G3'
		elif market in ('MAURITIUS','DUBAI, UNITED ARAB EMIRATES','AGADIR, MOROCCO','REUNION','ALGERIA','MALTA','SEYCHELLES','PATTAYA, THAILAND','AFRICA FR OTHER','MALDIVES','KRABI, THAILAND'):
			return 'G4'
		elif market in ('BANGKOK, THAILAND','CASABLANCA, MOROCCO','KOH PHI PHI, THAILAND','CHIANG MAI, THAILAND','JERUSALEM, ISRAEL'):
			return 'G5'
		else:
			return 'others'
		
	elif model_name == 'Europe1':
		if market == 'AMSTERDAM, NETHERLANDS':
			return 'AMSTERDAM, NETHERLANDS'
		elif market == 'REGIONAL FRANCE BRETAGNE':
			return 'REGIONAL FRANCE BRETAGNE'
		elif market == 'REGIONAL FRANCE NORMANDIE':
			return 'REGIONAL FRANCE NORMANDIE'
		elif market == 'REGIONAL FRANCE POITOU CHARENTE':
			return 'REGIONAL FRANCE POITOU CHARENTE'
		elif market in ('REGIONAL FRANCE ALSACE LORRAINE','REGIONAL FRANCE CORSICA','REGIONAL FRANCE PAYS DE LA LOIRE','REGIONAL FRANCE AQUITAINE','REGIONAL FRANCE MIDI PYRENEES','REGIONAL FRANCE LANGUEDOC ROUSSILLON','REGIONAL FRANCE LOURDES') :
			return 'G5'
		elif market in ('REGIONAL FRANCE NORD PICARDIE','REGIONAL FRANCE AUVERGNE LIMOUSIN','REGIONAL FRANCE RHONE ALPS','REGIONAL FRANCE CENTRE','REGIONAL FRANCE ILE DE FRANCE','REGIONAL FRANCE PROVENCE','REGIONAL FRANCE BOURGOGNE FRANCHE-COMTE'):
			return 'G6'
		elif market in ('BRUSSELS, BELGIUM','BRUGES, BELGIUM','WALLONIA, BELGIUM','ANTWERP, BELGIUM'):
			return 'G7'
		elif market in ('BERLIN, GERMANY','VIENNA, AUSTRIA','REGIONAL SPAIN SAN SEBASTIAN','BELGIUM COAST, BELGIUM'):
			return 'G8'
		elif market in ('GENEVA, SWITZERLAND','REGIONAL PORTUGAL CONTINENTAL','LUXEMBOURG, LUXEMBOURG','SWITZERLAND CITIES FRENCH/ITALIAN SPEAKING, SWITZERLAND','REGIONAL FRANCE CHAMPAGNE ARDENNE','MUNICH, GERMANY','REGIONAL GERMANY BLACK FOREST','REGIONAL SPAIN BILBAO','FRANKFURT, GERMANY','REGIONAL SPAIN CORDOBA','ROTTERDAM, NETHERLANDS','SWITZERLAND CITIES GERMAN SPEAKING, SWITZERLAND','REGIONAL UK SOUTH EAST') :
			return 'G9'
		else:
			return 'others'
		
	elif model_name=="Europe2":
		if market == 'LONDON':
			return 'LONDON'
		elif market == 'BARCELONA, SPAIN':
			return 'BARCELONA, SPAIN'
		elif market == 'ROME, ITALY':
			return 'ROME, ITALY'
		elif market == 'VENICE, ITALY':
			return 'VENICE, ITALY'
		elif market in ('GIRONA & COSTA BRAVA, SPAIN','ISTANBUL, TURKEY','LISBON, PORTUGAL','SALOU, SPAIN','LLORET DE MAR, SPAIN','MALLORCA ISLAND, SPAIN','PRAGUE, CZECH REPUBLIC','SARDINIA, ITALY','TENERIFE, SPAIN'):
			return 'G5'
		elif market in ('MILAN, ITALY','MADRID, SPAIN','FLORENCE, ITALY','SEVILLE, SPAIN','ANDORRA','PORTO, PORTUGAL','TARRAGONA & COSTA DORADA, SPAIN','RIVIERA DEI FIORI, ITALY','MALAGA, SPAIN','PALERMO (PROVINCE), ITALY','MESTRE, ITALY'):
			return 'G6'
		elif market in ('ALGARVE, PORTUGAL','ALBUFEIRA, PORTUGAL','SANTORINI, GREECE','BUDAPEST, HUNGARY','HERAKLION, CRETE','MARBELLA, SPAIN'):
			return 'G7'
		elif market in ('ATHENS, GREECE','TURIN, ITALY','TUSCANY CITIES, ITALY','VALENCIA, SPAIN','NAPLES, ITALY','GENOA CITY, ITALY','GRANADA, SPAIN','COMO LAKE, ITALY'):
			return 'G8'
		elif market in ('GRAN CANARIA, SPAIN','DALMATIA, CROATIA','IBIZA, SPAIN','MADEIRA ISLAND, PORTUGAL','RIMINI, ITALY','ALICANTE, SPAIN','GOLFO DEI POETI, ITALY','LANZAROTE, SPAIN','CHANIA, CRETE','ANTALYA, TURKEY','TALLINN, ESTONIA','BENIDORM, SPAIN','RHODES, GREECE','CYCLADES ISLANDS, GREECE','RETHIMNON, CRETE','LASITHI, CRETE'):
			return 'G9'
		elif market in ('MAGGIORE LAKE, ITALY','RIVIERA DELLE PALME, ITALY','GARDA LAKE, ITALY','LISBON COAST & TAGUS VALLEY, PORTUGAL','KRAKOW, POLAND','SKI WEST, ITALY','GOLFO TIGULLIO, ITALY'):
			return 'G10'
		else:
			return 'others'
		
	elif model_name=="Other Region":
		return 'NA'		

In [6]:
def srch_model_name(region):
	if region in ('FRANCE','PARIS'):
		return 'France'
		
	elif region in ('THAILAND','AFRICA AND EAST MED','MIDDLE EAST AND INDIAN OCEAN'):
		return 'TAM'
		
	elif region in ('EUROPE REGIONAL TERRITORIES','BENELUX','GERMANY, AUSTRIA & SWITZERLAND'):
		return 'Europe1'
		
	elif region in ('SPAIN & PORTUGAL','EASTERN EUROPE','ITALY NORTH','LONDON, ENGLAND','ROME & ITALY RESORTS','GREECE & TURKEY'):
		return 'Europe2'
		
	elif region in ('NEW YORK CITY','CARIBBEAN','UK & IRELAND','FLORIDA (USA)','GAMING (USA)',
                    'JAPAN & MICRONESIA','CALIFORNIA','OCEANIA','NORDIC','MEXICO & CENTRAL AMERICA',
                    'CANADA','NORTH AMERICA REGIONAL TERRITORIES','INDONESIA','SOUTH AMERICA',
                    'INDIAN SUBCONTINENT','SOUTHEAST ASIA EMERGING','MID-ATLANTIC','SF PACNW',
                    'MALAYSIA','MOUNTAIN (USA)','ORLANDO','MAINLAND CHINA','NORTHEAST (USA)',
                    'HAWAII','CENTRAL (USA)','HONG KONG & MACAU','MIDWEST (USA)','SINGAPORE (REGION)',
                    'PHILIPPINES','SOUTH KOREA','TAIWAN','UNKNOWN','HONG KONG REGION','MEXICO - DOMESTIC REGION',
                    'MACAU REGION','CENTRAL AMERICA REGION','ROME AND CENTRAL SOUTHERN ITALY','VIETNAM','AUSTRALIA',
                    'MEXICO - RESORTS REGION','NEW ZEALAND & PACIFIC ISLANDS'):
		return 'Other Region'

	else :
		return "NOT APPLICABLE"

In [7]:
def srch_region_bucket(region,model_name):
	if model_name in ('France','TAM','Europe1','Europe2'):
		return 'NA'
	if model_name == 'Other Region':
		if region in ('MAINLAND CHINA','SINGAPORE (REGION)','SOUTHEAST ASIA EMERGING','INDIAN SUBCONTINENT','SOUTH KOREA','MALAYSIA','JAPAN & MICRONESIA','OCEANIA','HONG KONG & MACAU','INDONESIA','PHILIPPINES','TAIWAN','UNKNOWN'):
			return 'G11'
		elif region in ('NEW YORK CITY','MOUNTAIN (USA)','MIDWEST (USA)','CALIFORNIA','GAMING (USA)','CENTRAL (USA)','FLORIDA (USA)','NORTHEAST (USA)','ORLANDO','SF PACNW','HAWAII'):
			return 'G21'
		elif  region in ('NORTH AMERICA REGIONAL TERRITORIES','SOUTH AMERICA','MEXICO & CENTRAL AMERICA','CANADA','CARIBBEAN'):
			return 'G31'
		elif region in ('MID-ATLANTIC','NORDIC','UK & IRELAND'):
			return 'G41'
		else:
			return 'Others'			
	else :
		return 'NA'

In [8]:
udfsrch_region_star_rating=udf(srch_region_star_rating, StringType())
udfsrch_region_hotel_type=udf(srch_region_Hotel_type, StringType())
udfsrch_region_market=udf(srch_region_market, StringType())
udfsrch_model_name= udf(srch_model_name,StringType())
udfsrch_region_bucket = udf (srch_region_bucket,StringType())

## Importing Hotel Dim

In [9]:
## Loading Hotel Dim table:
path_hotel_dim = 's3n://ewe-core-meta-prod/CORE/HOTEL_DIM/*'
hotel_dim = (sqlCtx.read.format("com.databricks.spark.csv")
            .options(header = False, inferSchema = True, delimiter = "\t", nullValue= '\\N')
            .load(path_hotel_dim)
            .distinct()
            .filter("C12 = 'BOOKABLE' or C12 ='NOT BOOKABLE' or C12 ='NOT APPLICABLE'")
            .select("C0","C7","C25","C26","C4","C13","C3")
            .withColumnRenamed("C0","HOTEL_ID")
            .withColumnRenamed("C7" , "city")
            .withColumnRenamed("C25","region")
            .withColumnRenamed("C26","market")
            .withColumnRenamed("C4","brand")
            .withColumnRenamed("C13","star_rating")
            .withColumnRenamed("C3","hotel_type")
            .filter("HOTEL_ID!='None'")
            )

In [10]:
## Finalizing Hotel Dim Table
hotel_dim_with_bkts = (hotel_dim.withColumn("MODEL_NAME", udfsrch_model_name("region"))
                        .withColumn("STAR_RATING_BUCKET", udfsrch_region_star_rating("star_rating"))
                        .withColumn("HOTEL_TYPE_BUCKET", udfsrch_region_hotel_type("hotel_type","MODEL_NAME"))
                        .withColumn("MARKET_BUCKET",udfsrch_region_market("market","MODEL_NAME"))
                        .withColumn("REGION_BUCKET", udfsrch_region_bucket("region","MODEL_NAME")))
                        

hotel_dim_final = (hotel_dim_with_bkts
                   .drop('city')
                   .drop('market')
                   .drop('brand')
                   .drop('star_rating')
                   .drop('hotel_type')
                   .repartition(1)
                   .cache())

## UDF for  Bucketing LOS and BW from Hotel Click Book Agg Table

In [11]:
##los bucket for search los
def searchToCategory(srch_los,):
    if srch_los in (0,1,2,3,4,5,6):
        return str(srch_los)
    else:
        return '>6'

##bw bucket for srch bw 
def searchbucketToCategory(SRCH_WINDOW):
    if SRCH_WINDOW <= 0:
        return '<=0'
    elif SRCH_WINDOW == 1:
        return '1'
    elif SRCH_WINDOW == 2:
        return '2'
    elif SRCH_WINDOW >= 3 and SRCH_WINDOW <= 4:
        return '3-4'
    elif SRCH_WINDOW >= 5 and SRCH_WINDOW <= 7:
        return '5-7'
    elif SRCH_WINDOW >= 8 and SRCH_WINDOW <= 14:
        return '8-14'
    elif SRCH_WINDOW >= 15 and SRCH_WINDOW <= 20:
        return '15-20'
    elif SRCH_WINDOW >= 21 and SRCH_WINDOW <= 30:
        return '21-30'
    elif SRCH_WINDOW >= 31 and SRCH_WINDOW <= 40:
        return '31-40'
    elif SRCH_WINDOW >= 41 and SRCH_WINDOW <= 60:
        return '41-60'
    elif SRCH_WINDOW >= 61 and SRCH_WINDOW <= 90:
        return '61-90'
    elif SRCH_WINDOW >= 91 and SRCH_WINDOW <= 120:
        return '91-120'
    elif SRCH_WINDOW >= 121 and SRCH_WINDOW <= 180:
        return '121-180'
    else:
        return '>180'
    


In [12]:
#los bucket for booked los
def srchBookedLosBucket(booked_los):
    if booked_los is None:
        return '0'
    elif booked_los  in (0,1,2,3,4,5,6):
        return str(booked_los)
    else:
        return '>6'

#bw bucket for booked bw
def srchBookedBwBucket(BOOKED_BKG_WINDOW):
    if BOOKED_BKG_WINDOW is None:
        return '0'
    elif BOOKED_BKG_WINDOW <= 0:
        return '<= 0'
    elif BOOKED_BKG_WINDOW == 1:
        return '1'
    elif BOOKED_BKG_WINDOW == 2:
        return '2'
    elif BOOKED_BKG_WINDOW >= 3 and BOOKED_BKG_WINDOW <= 4:
        return '3-4'
    elif BOOKED_BKG_WINDOW >= 5 and BOOKED_BKG_WINDOW <= 7:
        return '5-7'
    elif BOOKED_BKG_WINDOW >= 8 and BOOKED_BKG_WINDOW <= 14:
        return '8-14'
    elif BOOKED_BKG_WINDOW >= 15 and BOOKED_BKG_WINDOW <= 20:
        return '15-20'
    elif BOOKED_BKG_WINDOW >= 21 and BOOKED_BKG_WINDOW <= 30:
        return '21-30'
    elif BOOKED_BKG_WINDOW >= 31 and BOOKED_BKG_WINDOW <= 40:
        return '31-40'
    elif BOOKED_BKG_WINDOW >= 41 and BOOKED_BKG_WINDOW <= 60:
        return '41-60'
    elif BOOKED_BKG_WINDOW >= 61 and BOOKED_BKG_WINDOW <= 90:
        return '61-90'
    elif BOOKED_BKG_WINDOW >= 91 and BOOKED_BKG_WINDOW <= 120:
        return '91-120'
    elif BOOKED_BKG_WINDOW >= 121 and BOOKED_BKG_WINDOW <= 180:
        return '121-180'
    else:
        return '>180'

In [13]:
#converting booking flag from string values to boolean
def booking_flag(booked_flag):
    if booked_flag == 'Y':
        return 1
    else:
        return 0

In [14]:
# Registering Functions as UDF's
udfsearchToCategory=udf(searchToCategory, StringType())
udfsearchbucketToCategory=udf(searchbucketToCategory, StringType())
udfsrchBookedLosBucket=udf(srchBookedLosBucket, StringType())
udfsrchsrchBookedBwBucket=udf(srchBookedBwBucket, StringType())
booking_flag_udf=udf(booking_flag, IntegerType())

## Importing HCBA table

In [15]:
from datetime import *
import time

date_list = [
"2016-03-06",
"2016-03-07",
"2016-03-08",
"2016-03-09",
"2016-03-10",
"2016-03-11",
"2016-03-12",
"2016-03-13",
"2016-03-14",
"2016-03-15",
"2016-03-16",
"2016-03-17",
"2016-03-18",
"2016-03-19",
"2016-03-20",
"2016-03-21",
"2016-03-22",
"2016-03-23",
"2016-03-24",
"2016-03-25",
"2016-03-26",
"2016-03-27",
"2016-03-28",
"2016-03-29",
"2016-03-30",
"2016-03-31",
"2016-04-01",
"2016-04-02",
"2016-04-03",
"2016-04-04",
"2016-04-05",
"2016-04-06",
"2016-04-07",
"2016-04-08",
"2016-04-09",
"2016-04-10",
"2016-04-11",
"2016-04-12",
"2016-04-13",
"2016-04-14",
"2016-04-15",
"2016-04-16",
"2016-04-17",
"2016-04-18",
"2016-04-19",
"2016-04-20",
"2016-04-21",
"2016-04-22",
"2016-04-23",
"2016-04-24",
"2016-04-25",
"2016-04-26",
"2016-04-27",
"2016-04-28",
"2016-04-29",
"2016-04-30",
"2016-05-01",
"2016-05-02",
"2016-05-03",
"2016-05-04",
"2016-05-05",
"2016-05-06",
"2016-05-07",
"2016-05-08",
"2016-05-09",
"2016-05-10",
"2016-05-11",
"2016-05-12",
"2016-05-13",
"2016-05-14",
"2016-05-15",
"2016-05-16",
"2016-05-17",
"2016-05-18",
"2016-05-19",
"2016-05-20",
"2016-05-21",
"2016-05-22",
"2016-05-23",
"2016-05-24",
"2016-05-25",
"2016-05-26",
"2016-05-27",
"2016-05-28",
"2016-05-29",
"2016-05-30",
"2016-05-31",
"2016-06-01",
"2016-06-02",
"2016-06-03",
"2016-06-04",
"2016-06-05",
"2016-06-06",
"2016-06-07",
"2016-06-08",
"2016-06-09",
"2016-06-10",
"2016-06-11",
"2016-06-12",
"2016-06-13",
"2016-06-14",
"2016-06-15",
"2016-06-16",
"2016-06-17",
"2016-06-18",
"2016-06-19",
"2016-06-20",
"2016-06-21",
"2016-06-22",
"2016-06-23",
"2016-06-24",
"2016-06-25",
"2016-06-26",
"2016-06-27",
"2016-06-28",
"2016-06-29",
"2016-06-30",
"2016-07-01",
"2016-07-02",
"2015-06-28",
"2015-06-29",
"2015-06-30",
"2015-07-01",
"2015-07-02",
"2015-07-03",
"2015-07-04",


]

# start_date = datetime.strptime("2016-05-22", "%Y-%m-%d")
# for i in range(91):
#    x = (start_date - timedelta(i))
#    x = x.strftime("%Y-%m-%d")
#    date_list.append(x)

#schema for HCBA table
hcba_schema = StructType([
                            StructField('PARTNER_POS',StringType(), True),
                            StructField('DEVICE_TYPE',StringType(), True),
                            StructField('CLICKED_HOTEL_ID',IntegerType(), True),
                            StructField('SRCH_WINDOW',IntegerType(), True),
                            StructField('SRCH_LOS',IntegerType(), True),
                            StructField('TRAFFIC',IntegerType(), True),
                            StructField('COST',DoubleType(), True),
                            StructField('BOOKED_FLAG',StringType(), True),
                            StructField('BOOKED_HOTEL_ID',IntegerType(), True),
                            StructField('BOOKED_LOS',IntegerType(), True),
                            StructField('BOOKED_BKG_WINDOW',IntegerType(), True),
                            StructField('GROSS_PROFIT',DoubleType(), True),
                            StructField('PARTNER_ORG',StringType(), True),
                            StructField('ACTIVITY_DATE',StringType(), True)
                          ])

##CREATE EMPTY DATAFRAME 
hcba = sqlContext.createDataFrame(sc.emptyRDD(), hcba_schema)

In [16]:
##APPENDING THE DATA IN ABOVE CREATED hcba TABLE 
for dates in date_list:
    path_hcba = 's3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date='+dates+'/*'  
    print(path_hcba)
    try :
        temp_hcba = (sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferschema = True, delimiter = "\t",nullValue= '\\N')
                    .load(path_hcba)
                    .select('C1','C3','C7','C8','C9','C13','C14','C19','C20','C21','C22','C45','C65')
                    .filter("C65 = 'TRIVAGO' and C1 = 'FR' and C7 != '0' and C13 != '0'") 
                    .withColumn('ACTIVITY_DATE',lit(dates))
                    )
        hcba = hcba.unionAll(temp_hcba)
        print('file exists')
    except: 
#         print (path_hcba)
        print('file does not exist')

s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-06/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-07/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-08/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-09/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-10/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-11/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-12/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-13/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-14/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-15/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-03-16/*
file exists
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_B

## Cleaning and Outlier Treatment of HCBA data

In [17]:
## Changing Data Type of HCBA Table
cols_int = ['SRCH_LOS','SRCH_WINDOW','TRAFFIC','BOOKED_BKG_WINDOW','BOOKED_LOS']
cols_float = ['COST','GROSS_PROFIT']

hcba_changed_data_type = data_type_change_float(data_type_change_int(hcba,cols_int),cols_float)

hcba_final = (hcba_changed_data_type
                .withColumn("SRCH_LOS_BUCKET", udfsearchToCategory("SRCH_LOS"))
                .withColumn("SRCH_BW_BUCKET", udfsearchbucketToCategory("SRCH_WINDOW"))
                .withColumn("BOOKED_LOS_BUCKET", udfsrchBookedLosBucket("BOOKED_LOS"))
                .withColumn("BOOKED_BW_BUCKET", udfsrchsrchBookedBwBucket("BOOKED_BKG_WINDOW"))
                .withColumn("BOOKING", booking_flag_udf("BOOKED_FLAG"))
                .repartition(1)
             )

#columns required in final cleaned hcba table
cols_required = ['PARTNER_POS','DEVICE_TYPE','HOTEL_ID','PARTNER_ORG','ACTIVITY_DATE','TRAFFIC','BOOKING_WINDOW','LOS',
                 'COST','GROSS_PROFIT','LOS_BUCKET','BW_BUCKET','BOOKING']

#replacing the wrongly populated search parameters for the search queries that are booked with booking parameters
hcba_replac_srch_with_booking = (hcba_final.filter("BOOKED_FLAG = 'Y'")
                                 .filter("SRCH_WINDOW < -1 or SRCH_LOS <= 0")
                                 .withColumnRenamed('BOOKED_BKG_WINDOW', 'BOOKING_WINDOW')
                                 .withColumnRenamed('BOOKED_BW_BUCKET', 'BW_BUCKET')
                                 .withColumnRenamed('BOOKED_LOS_BUCKET', 'LOS_BUCKET')
                                 .withColumnRenamed('BOOKED_HOTEL_ID', 'HOTEL_ID')
                                 .withColumnRenamed('BOOKED_LOS', 'LOS')
                                 .select(cols_required)
                            )

#filtering out the outliers from the seacrh data
hcba_srch_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'N' and SRCH_WINDOW >= -1 and SRCH_LOS > 0 and GROSS_PROFIT = 0.0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )

#filtering out the outilers for the booking data
hcba_booking_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'Y' and SRCH_WINDOW >= -1 and SRCH_LOS > 0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )



In [18]:
#Union of all the above 3 tables 
hcba_cleaned_data =( hcba_replac_srch_with_booking.unionAll(hcba_srch_data_cleaned)
                    .unionAll(hcba_booking_data_cleaned)
                    .repartition(1)
                    .cache())

## Subset HCBA for populating avg weekly traffic

In [19]:
hcba_opti = (hcba_cleaned_data.filter("ACTIVITY_DATE >= '2016-01-01'").filter("ACTIVITY_DATE <= '2016-06-25'")# valid 1
                   .withColumn('WEEK',weekofyear("ACTIVITY_DATE"))) 

hcba_opti_agg = hcba_opti.groupBy('HOTEL_ID','LOS_BUCKET','BW_BUCKET')

hcba_opti_agg_total_traffic = (hcba_opti_agg.agg({'TRAFFIC':'sum','BOOKING':'sum','GROSS_PROFIT':'sum','COST':'sum'})
                                .withColumnRenamed('sum(TRAFFIC)','TRAFFIC')
                                .withColumnRenamed('sum(BOOKING)','BOOKING')
                                .withColumnRenamed('sum(GROSS_PROFIT)','GROSS_PROFIT')
                                .withColumnRenamed('sum(COST)','COST')
                                .repartition(1))

hcba_opti_agg_number_weeks =( hcba_opti_agg.agg(func.countDistinct('WEEK'))
                            .withColumnRenamed('count(WEEK)','NO_WEEKS')
                            .repartition(1))

hcba_opti_with_cols_traffic_weeks =(hcba_opti_agg_total_traffic
                                    .join(hcba_opti_agg_number_weeks,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'inner')
                                    .repartition(1))

hcba_opti_final1 = (hcba_opti_with_cols_traffic_weeks.withColumn('AVERAGE_WEEKLY_TRAFFIC',
                    hcba_opti_with_cols_traffic_weeks.TRAFFIC/hcba_opti_with_cols_traffic_weeks.NO_WEEKS)
                    .withColumn('CPC_HCBA',
                    hcba_opti_with_cols_traffic_weeks.COST/hcba_opti_with_cols_traffic_weeks.TRAFFIC)
                   .groupBy('HOTEL_ID','LOS_BUCKET','BW_BUCKET')
                   .agg({'AVERAGE_WEEKLY_TRAFFIC':'avg','CPC_HCBA':'avg'})
                   .withColumnRenamed("avg(AVERAGE_WEEKLY_TRAFFIC)","AVERAGE_WEEKLY_TRAFFIC")
                   .withColumnRenamed("avg(CPC_HCBA)","CPC_HCBA")
                   .drop("DEVICE_TYPE"))

hcba_opti_final = (hcba_opti_final1
                  .repartition(1)
                  .cache())

## Creating the Training AD for RF using HCBA and not MBAP Tables

In [25]:
# splitting HCBA for training and validation
hcba_rf_union_train1 = hcba_cleaned_data.filter("ACTIVITY_DATE >= '2016-06-05' and ACTIVITY_DATE <= '2016-06-25'")
hcba_rf_union_train2 = hcba_cleaned_data.filter("ACTIVITY_DATE >= '2015-06-28'").filter("ACTIVITY_DATE <= '2015-07-04'")
hcba_rf_union_train = hcba_rf_union_train1.unionAll(hcba_rf_union_train2)
hcba_rf_union_valid = hcba_cleaned_data.filter("ACTIVITY_DATE >= '2016-06-26'")

In [26]:
rf_training_data = (hcba_rf_union_train.join(hotel_dim_final,"HOTEL_ID","inner")
                        .repartition(1)
                        .cache()
                   )

rf_validation_data = (hcba_rf_union_valid.join(hotel_dim_final,"HOTEL_ID","inner")
                        .repartition(1)
                        .cache()
                     )

In [27]:
rf_training_data = (rf_training_data.withColumn("NEW_MARKET_BUCKET",
                            when(rf_training_data.MODEL_NAME=="Other Region",rf_training_data.REGION_BUCKET)
                            .otherwise(rf_training_data.MARKET_BUCKET))
                            .repartition(1)
                            .cache()
                         )

rf_validation_data = (rf_validation_data.withColumn("NEW_MARKET_BUCKET",
                            when(rf_validation_data.MODEL_NAME=="Other Region",rf_validation_data.REGION_BUCKET)
                            .otherwise(rf_validation_data.MARKET_BUCKET))
                            .repartition(1)
                            .cache()
                         )

## Importing EPB Master table (If Net Revenue then TOP_HOTEL_PRICE_AGG)


In [28]:
###change the table that needs to imported after finalising the approach
opti_search_query_universe = (sqlContext.read.parquet("s3n://ewe-meta-top/EPB_TABLES_TRIVAGO_FR/combine_desktop_mobile/3_month_collect/sq_univ_trivagoFR_vweek_06262016")
                              .select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","EPB")
                              .repartition(4,"HOTEL_ID","LOS_BUCKET","BW_BUCKET")
                              .cache())

# Creating Optimization AD

In [29]:
# Joining optimization universe with EPB field and HCBA for average weekly traffic and hotel_dim for hotel dimensions
opti_search_query_universe_final1 = (opti_search_query_universe[opti_search_query_universe.EPB.isNotNull()]
                                                  .join(hcba_opti_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],'left_outer')
                                                  .join(hotel_dim_final,"HOTEL_ID","inner")
                                                  )

In [30]:
opti_search_query_universe_final = (opti_search_query_universe_final1
                                    .withColumn("NEW_MARKET_BUCKET"
                                    ,when(opti_search_query_universe_final1.MODEL_NAME=="Other Region",opti_search_query_universe_final1.REGION_BUCKET)
                                    .otherwise(opti_search_query_universe_final1.MARKET_BUCKET))
                                    .repartition(4)
                                    .cache())

## Random Forest Model Generation

In [31]:
from pyspark.mllib.linalg import SparseVector, VectorUDT
import numpy
from pyspark.ml.feature import *
from pyspark.mllib.linalg import *
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer

In [32]:
region_level = 'NEW_MARKET_BUCKET'

In [33]:
training_ad = (rf_training_data
               .withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET'))

optimization_test = (opti_search_query_universe_final
                     .withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET'))

In [34]:
# # adding code for valdiation
validation_ad = (rf_validation_data.withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET'))

In [35]:
# for training the data
# Aggregating the data to the required level
col_agg = ["BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET", "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET"]
training_ad_T = (training_ad.groupBy(col_agg)
                      .sum("TRAFFIC", "BOOKING","COST","GROSS_PROFIT")
                      .withColumnRenamed('sum(TRAFFIC)', 'TRAFFIC')
                      .withColumnRenamed('sum(BOOKING)', 'BOOKING')
                      .withColumnRenamed('sum(COST)', 'COST')
                      .withColumnRenamed('sum(GROSS_PROFIT)', 'GROSS_PROFIT'))
validation_ad_T = (validation_ad.groupBy(col_agg)
                      .sum("TRAFFIC", "BOOKING","COST","GROSS_PROFIT")
                      .withColumnRenamed('sum(TRAFFIC)', 'TRAFFIC')
                      .withColumnRenamed('sum(BOOKING)', 'BOOKING')
                      .withColumnRenamed('sum(COST)', 'COST')
                      .withColumnRenamed('sum(GROSS_PROFIT)', 'GROSS_PROFIT'))

In [36]:
# Filtering out the TRAFFIC (Outliers) from data
training_ad_T1 = (training_ad_T[~((training_ad_T.TRAFFIC <= 4) & (training_ad_T.BOOKING > 0))]
                               [~((training_ad_T.TRAFFIC == 5) & (training_ad_T.BOOKING == 2))]
                               [~((training_ad_T.TRAFFIC == 6) & (training_ad_T.BOOKING == 2))])

In [37]:
col_required_rf = ["MODEL_NAME","HOTEL_ID","LOS_BUCKET","BW_BUCKET","DEVICE_TYPE","STAR_RATING_BUCKET",
                   "HOTEL_TYPE_BUCKET","REGION_LEVEL_BUCKET","AVERAGE_WEEKLY_TRAFFIC"
                   ,"EPB","TRAFFIC","BOOKING","chk_data","CPC_HCBA","GROSS_PROFIT","COST",'ACTIVITY_DATE']

In [38]:
training_ad_T2 = (training_ad_T1.withColumn("chk_data",lit("train"))
                                .withColumn('MODEL_NAME',lit("all"))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP_MOBILE"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('CPC',lit(0.0))
                                .withColumn('BASE_RATE_MIN',lit(0.00))
                                .withColumn('EPB',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_rf))

In [39]:
optimization_test2 = (optimization_test.withColumn('BASE_RATE_MIN',lit(0.00))
                                      .withColumn('TRAFFIC',lit(0))
                                      .withColumn('BOOKING',lit(0))
                                      .withColumn('GROSS_PROFIT',lit(0.0))
                                      .withColumn('COST',lit(0.0))
                                      .withColumn("DEVICE_TYPE",lit("DESKTOP_MOBILE"))
                                      .withColumn("chk_data",lit("test"))
                                      .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                      .select(col_required_rf))

In [40]:
training_for_test = training_ad_T
validation_for_test = validation_ad_T 

In [41]:
col_required_results = ["MODEL_NAME","HOTEL_ID","LOS_BUCKET","BW_BUCKET","DEVICE_TYPE","STAR_RATING_BUCKET",
                   "HOTEL_TYPE_BUCKET","REGION_LEVEL_BUCKET","AVERAGE_WEEKLY_TRAFFIC", 
                        "EPB","TRAFFIC","BOOKING","chk_data","CPC_HCBA","GROSS_PROFIT","COST",'ACTIVITY_DATE']

In [42]:
# for testing on training data
training_for_test1 = (training_for_test.withColumn("chk_data",lit("train-testing"))
                                .withColumn('MODEL_NAME',lit('name'))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP_MOBILE"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('CPC',lit(0.0))
                                .withColumn('BASE_RATE_MIN',lit(0.00))
                                .withColumn('EPB',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_results))

In [43]:
# for testing on validation data
validation_for_test1 = (validation_for_test.withColumn("chk_data",lit("validation-testing"))
                                .withColumn('MODEL_NAME',lit('name'))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP_MOBILE"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('CPC',lit(0.0))
                                .withColumn('BASE_RATE_MIN',lit(0.00))
                                .withColumn('EPB',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_results))

In [44]:
test_train_data = (optimization_test2.unionAll(training_ad_T2)
                                    .unionAll(training_for_test1)
                                    .unionAll(validation_for_test1)
                                    .repartition(15)
                                    .cache())

In [45]:
# STRING INDEXING: Converting the data type of required columns to factor
def indexStringColumns(df, cols):
    tempdf = df
    for col in cols:
        stringIndexer = StringIndexer(inputCol=col, outputCol=col + "-num")
        si_model = stringIndexer.fit(tempdf)
        tempdf = si_model.transform(tempdf)
    return tempdf

In [46]:
cols_rf_operations = {"BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET","STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET"}
dfnumeric = indexStringColumns(test_train_data, cols_rf_operations)

In [47]:
# One HOT Encoding
def oneHotEncodeColumns(df, cols):
    tempdf = df
    for col in cols:
        onehotenc = OneHotEncoder(
            inputCol=col + "-num", outputCol=col + "-onehot")
        tempdf = onehotenc.transform(tempdf).drop(col + "-num")
    return tempdf

In [48]:
test_train_data_class = oneHotEncodeColumns(dfnumeric, cols_rf_operations)

In [49]:
cols_rf_model = ["BW_BUCKET","LOS_BUCKET","HOTEL_TYPE_BUCKET","STAR_RATING_BUCKET","REGION_LEVEL_BUCKET", 
                "TRAFFIC","BOOKING","STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot", 
                "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot"]

In [50]:
training_ad_T1_Class = (test_train_data_class.filter(test_train_data_class.chk_data=="train")
                        .drop("chk_data")
                        .select(cols_rf_model))

In [51]:
# Transforming the training dataset
# Defining function for the same
def genRows(row):
    TRAFFIC = row[5]
    BOOKING = row[6]
    BOOKINGcounter = 0
    rowList = []

    for i in range(TRAFFIC):
        row1 = row[:]
        row1 = list(row1)
        row1[5] = 1
        row1 = tuple(row1)
        if BOOKINGcounter < int(BOOKING):
            row1 = list(row1)
            row1[6] = 1
            row1 = tuple(row1)
        else:
            row1 = list(row1)
            row1[6] = 0
            row1 = tuple(row1)
        BOOKINGcounter += 1
        rowList.append(row1)
    return rowList

In [52]:
training_ad_T1_Class_rdd = training_ad_T1_Class.rdd
training_ad_T1_Class_rdd = training_ad_T1_Class_rdd.flatMap(lambda x: genRows(x))

In [53]:
# Reconverting to dataframe
from pyspark.mllib.linalg import SparseVector, VectorUDT

schema = StructType([StructField('BW_BUCKET', StringType(), True),
                     StructField('LOS_BUCKET', StringType(), True),
                     StructField('HOTEL_TYPE_BUCKET', StringType(), True),
                     StructField('STAR_RATING_BUCKET', StringType(), True),
                     StructField('REGION_LEVEL_BUCKET',StringType(), True),
                     StructField('TRAFFIC', IntegerType(), True),
                     StructField('BOOKING', IntegerType(), True),
                     StructField('HOTEL_TYPE_BUCKET-onehot',VectorUDT(), True),
                     StructField('BW_BUCKET-onehot',  VectorUDT(), True),
                     StructField('REGION_LEVEL_BUCKET-onehot',VectorUDT(), True),
                     StructField('LOS_BUCKET-onehot',  VectorUDT(), True),
                     StructField('STAR_RATING_BUCKET-onehot',  VectorUDT(), True)])

training_ad_T1_Class_df = sqlContext.createDataFrame(training_ad_T1_Class_rdd, schema)

In [54]:
# Converting label to double datatype
toDouble = udf(lambda x: float(x * 1.00), DoubleType())

training_ad_T1_Class_df = training_ad_T1_Class_df.withColumn('BOOKING', toDouble(training_ad_T1_Class_df.BOOKING))

In [55]:
# Taking only relevant columns as features
colList = training_ad_T1_Class_df.columns
colList.remove('BOOKING')
colList.remove('TRAFFIC')
colList.remove('BW_BUCKET')
colList.remove('LOS_BUCKET')
colList.remove('HOTEL_TYPE_BUCKET')
colList.remove('STAR_RATING_BUCKET')
colList.remove('REGION_LEVEL_BUCKET')

vecAssembler = VectorAssembler(inputCols=colList, outputCol="features")

cols_train_ad_rf = ["BOOKING", "TRAFFIC", "BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET",
                    "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET", "features"]

train_ad = (vecAssembler.transform(training_ad_T1_Class_df)
                        .select(cols_train_ad_rf)
                        .withColumnRenamed("BOOKING", "label")
                        .repartition(1)
                        .cache())

In [56]:
col_remove = ["STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot",
              "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot"]

optimization_onehot = (test_train_data_class.filter(test_train_data_class.chk_data=="test")
                                            .drop("chk_data"))

vecAssembler = VectorAssembler(inputCols=colList, outputCol="indexedFeatures")

optimization_df_onehot = vecAssembler.transform(optimization_onehot)
                          
optimization_df = (optimization_df_onehot.select([col for col in optimization_df_onehot.columns if col not in col_remove])
                  .repartition(50)
                  .cache())

In [57]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train_ad)

featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(train_ad)

rf = RandomForestRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures", featureSubsetStrategy="all",
                           numTrees=1000, impurity="variance", maxBins=32,maxDepth=4)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

model = pipeline.fit(train_ad)

RF_Model = model.stages[2]

In [58]:
optimization_df_predict = (RF_Model.transform(optimization_df)
                           .drop("indexedFeatures")
                           .withColumnRenamed("prediction", "CONVERSION_RATE"))

In [59]:
region_df_for_opti = (optimization_df_predict 
                      .cache())

# Optimization in Loop

In [60]:
import pulp
from pulp import *

def df_create(x):
    mydictionary = x['var_list']
    Efficiency =  x['cnt']
    Model_Name = x['region_name'][0]
    
    var_list = []
    for key in mydictionary:
        var_list.append((key, mydictionary[key],Efficiency,Model_Name))
          
    df = pandas.DataFrame(var_list, columns=['var_id', 'var_value','Efficiency','Model'])
    df_final = sqlContext.createDataFrame(df)
    
    return df_final 
	
# here is where nathan is picking up on the code, going to include both training and validation sets
def threshold(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (1-z)

threshold_udf = udf(threshold,DoubleType())

def traffic_allow(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (z)

traffic_allow_udf = udf(traffic_allow,DoubleType())

#if conversion rate is less then 0.0005 then taking it as 0
def predicted(predicted):
    if predicted < 0.0005:
        predicted = 0.0
    else:
        predicted
    return predicted

##If Net revenue is negative then making it positive 
def net_revenue(net_revenue):
    if net_revenue < 0.0 :
        net_revenue = net_revenue*(-1)
    else:
        net_revenue
    return net_revenue

##creating unique id for each search query
def var_name(id):
    return "T_" + str(id)
	
##Creating the objective function at search query level, combining it while running lp_solve algorithm
def objective_fun(net_revenue,predicted):
    return net_revenue * predicted
	

##registering the above defined functions as uoptimization_rf
objective_fun_udf = udf(objective_fun, DoubleType())
var_name_udf = udf(var_name,StringType())
net_revenue_udf = udf(net_revenue, DoubleType())
predicted_udf = udf(predicted , DoubleType())

#Filtering rows with Traffic > 0 
optimization_rf_algo = region_df_for_opti[~region_df_for_opti.AVERAGE_WEEKLY_TRAFFIC.isNull()]

#Applying Prediction UDF for rounding CR less than 0.0005
region_df_algo_treated = (optimization_rf_algo.withColumn("predicted_new",predicted_udf("CONVERSION_RATE"))
                            .withColumn("net_revenue_new",net_revenue_udf("EPB")))

####change the limit ranges if required
constraint_list = [[0.2,0.2,0.2,0.2],	[0.2,0.5,0.2,0.5],	[0.8,0.2,0.2,0.5],	[0.2,0.2,0.2,0.8],	[0.8,0.5,0.5,0.3]]



counter = 1

for l1 in constraint_list:

    def lower_limit_fun(net_revenue,average_weekly_traffic):
        if net_revenue >= 100.00:
            limit = l1[0]*average_weekly_traffic
        elif (net_revenue >= 50.00) and (net_revenue < 100):
            limit = l1[1]*average_weekly_traffic
        elif (net_revenue >= 20.00) and (net_revenue < 50):
            limit = l1[2]*average_weekly_traffic
        else:
            limit=l1[3]*average_weekly_traffic

        return limit


    lower_limit_fun_udf = udf(lower_limit_fun, DoubleType())


    region_df_algo_final= (region_df_algo_treated.withColumn("objective_fun",objective_fun_udf("net_revenue_new" ,"predicted_new"))
                                .withColumn("Lower_Limit", lower_limit_fun_udf("net_revenue_new" , "AVERAGE_WEEKLY_TRAFFIC"))
                                .withColumn("id", monotonicallyIncreasingId())
                                .withColumn("var_name" ,var_name_udf("id"))
                                .rdd
                                .repartition(1)
                                .cache())

    region_count = region_df_algo_final.count()


    ##list of variuable names for opti algo
    t_list = region_df_algo_final.map(lambda x  : x.var_name).collect()

    ##objective function for lp solve
    objective_list =  region_df_algo_final.map(lambda x  : x.objective_fun).collect()
    final_objective = {t_list[i]: objective_list[i] for i in range(0, region_count)}

    ##lower and upper limit 
    lower_limit_list =  region_df_algo_final.map(lambda x  : x.Lower_Limit).collect()
    upper_limit_list =  region_df_algo_final.map(lambda x  : x.AVERAGE_WEEKLY_TRAFFIC).collect()
    lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, region_count)}
    upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, region_count)}


    ## Optimization Algorithm
    cnt = 0.1
    lp_status = "Start"
    region_name = region_df_algo_final.first()[0]

    while ((lp_status != "Optimal")  & (cnt <=5.0)):
        a = cnt

        def constraint_fun(row):
            last_row = row.CPC_HCBA - a * row.objective_fun
            return last_row

        last_row = region_df_algo_final.map(lambda x: constraint_fun(x)).collect()

        final_constraint = {t_list[i]: last_row[i] for i in range(0, region_count)}

        prob = LpProblem("The Optimization Problem", LpMaximize)

        lp_var = {}
        for variable in t_list:
            variable = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
            lp_var[str(variable)] = variable

        prob += lpSum([final_objective[i]*lp_var[i] for i in t_list])

        prob += lpSum([final_constraint[i]*lp_var[i] for i in t_list]) == 0

        prob.solve()

        lp_status = LpStatus[prob.status]

        var_values = {"start":0}
        for v in prob.variables():
            var_values.update({v.name[2:]:v.varValue})

        results={"region_name":region_name,"lp_status":LpStatus[prob.status],"cnt":(cnt), "gp":value(prob.objective),"var_list":var_values}

        cnt += 0.1

    final_results= [results]



    import pandas
    region_df_algo_final = sqlContext.createDataFrame(region_df_algo_final) #changing back to sqlcontext

    field = [StructField("var_id", StringType(), True),StructField("var_value", DoubleType(), True),StructField("Efficiency", FloatType(), True),StructField("Model", StringType(), True)]
    schema = StructType(field)

    optimization_df_algo1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)

    for i in range(len(final_results)):
        optimization_df_algo1 = optimization_df_algo1.unionAll(df_create(final_results[i]))

    optimization_df_algo1 = optimization_df_algo1.cache()
    

    optimization_df_algo1 = optimization_df_algo1.withColumn("id",optimization_df_algo1["var_id"].cast(LongType()))
    optimization_df_algo2 = region_df_algo_final.select([col for col in region_df_algo_final.columns if col not in {"predicted_new","net_revenue_new","objective_fun","Lower_Limit","var_name","part_id"}]).join(optimization_df_algo1,"id","inner")



    optimization_df_algo3 = optimization_df_algo2.withColumn("threshold",threshold_udf("var_value","AVERAGE_WEEKLY_TRAFFIC")).                        withColumn("traffic_allow",traffic_allow_udf("var_value","AVERAGE_WEEKLY_TRAFFIC"))

    optimization_df_algo3 = optimization_df_algo3.select([col for col in optimization_df_algo3.columns if col not in {"id","var_id","Model"}])
    optimization_df_algo3 = optimization_df_algo3.withColumnRenamed("var_value","Lp_solution")
    optimization_df_algo_final = optimization_df_algo3.select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","threshold","EPB","traffic_allow")

    hcba_updatedcols = (hcba_opti_final.select('HOTEL_ID','LOS_BUCKET','BW_BUCKET','CPC_HCBA','AVERAGE_WEEKLY_TRAFFIC')
                                    .withColumnRenamed('AVERAGE_WEEKLY_TRAFFIC','AVERAGE_WEEKLY_TRAFFIC-usedintraining')
                                    .withColumnRenamed('CPC_HCBA','CPC_HCBA-usedintraining'))

    
    validation_df_spark = validation_ad.join(hcba_updatedcols,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'left_outer') #changing so that we have the predicted conversion rate as well

    
    common_queries = validation_df_spark.join(optimization_df_algo_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

    valid_df = common_queries.withColumn("throttle_traffic",(common_queries.traffic_allow*common_queries.TRAFFIC))

    valid_df = valid_df.withColumn("throttle_cost",((valid_df.COST*valid_df.throttle_traffic)/valid_df.TRAFFIC))
    valid_df = valid_df.withColumn("average_booking",(valid_df.BOOKING*valid_df.traffic_allow))
    valid_df = valid_df.withColumn("average_earning",((valid_df.GROSS_PROFIT*valid_df.average_booking)/valid_df.BOOKING))

    const = str(l1[0])+"_"+str(l1[1])+"_"+str(l1[2])+"_"+str(l1[3])

    if counter == 1: #change it to the first number from which starting to run
        valid_actual=valid_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"})
       
        valid_throttle = (valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
                            .withColumn("constraint",lit(const)))

    else :
        valid_throttle = valid_throttle.unionAll((valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
                                                 .withColumn("constraint",lit(const))))


    counter += 1
    print(counter)



2
3
4
5
6


In [61]:
valid_throttle.show(10)

+---------------------+------------------+--------------------+--------------------+---------------+
|sum(throttle_traffic)|sum(throttle_cost)|sum(average_earning)|sum(average_booking)|     constraint|
+---------------------+------------------+--------------------+--------------------+---------------+
|    9872.999998306124| 4416.715022304814|   2741.205908183334|   41.80000002296455|0.2_0.2_0.2_0.2|
|   13883.199998952596| 6280.893920914331|    3482.34767862765|   67.10000004394284|0.2_0.5_0.2_0.5|
|   13937.699998989558| 6294.433449323689|   3482.347678627651|   67.10000004394283|0.8_0.2_0.2_0.5|
|   15613.999999209374| 7100.204643216482|  3813.1793686276496|   83.60000004394279|0.2_0.2_0.2_0.8|
|   13866.099998803347| 6326.317331301081|  3495.7975291637035|  61.300000051605124|0.8_0.5_0.5_0.3|
+---------------------+------------------+--------------------+--------------------+---------------+



In [62]:
valid_actual.show()

+------------+-----------------+-----------------+------------+
|sum(TRAFFIC)|        sum(COST)|sum(GROSS_PROFIT)|sum(BOOKING)|
+------------+-----------------+-----------------+------------+
|       19805|9265.864797694147|        4666.8851|         109|
+------------+-----------------+-----------------+------------+



In [63]:
validation_df_spark.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"}).show()

+------------+------------------+------------------+------------+
|sum(TRAFFIC)|         sum(COST)| sum(GROSS_PROFIT)|sum(BOOKING)|
+------------+------------------+------------------+------------+
|       37131|16687.656803190686|10421.844800000004|         221|
+------------+------------------+------------------+------------+



## Optimization without loop

## UDF for Variable Name, Conversion Rate and Net Revenue Treatment

In [ ]:
import pulp
from pulp import *

In [312]:
#if conversion rate is less then 0.0005 then taking it as 0
def predicted(predicted):
    if predicted < 0.0005:
        predicted = 0.0
    else:
        predicted
    return predicted

##If Net revenue is negative then making it positive 
def net_revenue(net_revenue):
    if net_revenue < 0.0 :
        net_revenue = net_revenue*(-1)
    else:
        net_revenue
    return net_revenue

##creating unique id for each search query
def var_name(id):
    return "T_" + str(id)

## UDF for Objective Function and Lower Limit for LP Solve

In [313]:
##Creating the objective function at search query level, combining it while running lp_solve algorithm
def objective_fun(net_revenue,predicted):
    return net_revenue * predicted

##Lower limit for the average weekly Traffic
def lower_limit_fun(net_revenue,average_weekly_traffic):
    if net_revenue >= 100.00:
        limit = 0.8*average_weekly_traffic
    elif net_revenue >= 50.00 and net_revenue < 100:
        limit = 0.5*average_weekly_traffic
    elif net_revenue >= 20.00 and net_revenue < 50:
        limit = 0.5*average_weekly_traffic
    else:
        limit=0.3*average_weekly_traffic
        
    return limit

In [314]:
##registering the above defined functions as uoptimization_rf
objective_fun_udf = udf(objective_fun, DoubleType())
lower_limit_fun_udf = udf(lower_limit_fun, DoubleType())
var_name_udf = udf(var_name,StringType())
net_revenue_udf = udf(net_revenue, DoubleType())
predicted_udf = udf(predicted , DoubleType())

In [315]:
#Filtering rows with Traffic > 0 
optimization_rf_algo = region_df_for_opti[~region_df_for_opti.AVERAGE_WEEKLY_TRAFFIC.isNull()]

#Applying Prediction UDF for rounding CR less than 0.0005
region_df_algo_treated = (optimization_rf_algo.withColumn("predicted_new",predicted_udf("CONVERSION_RATE"))
                            .withColumn("net_revenue_new",net_revenue_udf("EPB")))

region_df_algo_final= (region_df_algo_treated.withColumn("objective_fun",objective_fun_udf("net_revenue_new" ,"predicted_new"))
                            .withColumn("Lower_Limit", lower_limit_fun_udf("net_revenue_new" , "AVERAGE_WEEKLY_TRAFFIC"))
                            .withColumn("id", monotonicallyIncreasingId())
                            .withColumn("var_name" ,var_name_udf("id"))
                            .rdd
                            .repartition(1)
                            .cache())

region_count = region_df_algo_final.count()

KeyboardInterrupt: 

In [ ]:
##list of variuable names for opti algo
t_list = region_df_algo_final.map(lambda x  : x.var_name).collect()

##objective function for lp solve
objective_list =  region_df_algo_final.map(lambda x  : x.objective_fun).collect()
final_objective = {t_list[i]: objective_list[i] for i in range(0, region_count)}

##lower and upper limit 
lower_limit_list =  region_df_algo_final.map(lambda x  : x.Lower_Limit).collect()
upper_limit_list =  region_df_algo_final.map(lambda x  : x.AVERAGE_WEEKLY_TRAFFIC).collect()
lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, region_count)}
upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, region_count)}

In [ ]:
## Optimization Algorithm
cnt = 0.1
lp_status = "Start"
region_name = region_df_algo_final.first()[0]

while ((lp_status != "Optimal")  & (cnt <=5.0)):
    a = cnt
    
    def constraint_fun(row):
        last_row = row.CPC_HCBA - a * row.objective_fun
        return last_row
    
    last_row = region_df_algo_final.map(lambda x: constraint_fun(x)).collect()
    
    final_constraint = {t_list[i]: last_row[i] for i in range(0, region_count)}
    
    prob = LpProblem("The Optimization Problem", LpMaximize)
    
    lp_var = {}
    for variable in t_list:
        variable = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
        lp_var[str(variable)] = variable
    
    prob += lpSum([final_objective[i]*lp_var[i] for i in t_list])
    
    prob += lpSum([final_constraint[i]*lp_var[i] for i in t_list]) == 0
    
    prob.solve()
    
    lp_status = LpStatus[prob.status]
    
    var_values = {"start":0}
    for v in prob.variables():
        var_values.update({v.name[2:]:v.varValue})
   
    results={"region_name":region_name,"lp_status":LpStatus[prob.status],"cnt":(cnt), "gp":value(prob.objective),"var_list":var_values}
    
    cnt += 0.1
    
    print(cnt)

In [ ]:
results

In [ ]:
final_results= [results]

In [ ]:
def df_create(x):
    mydictionary = x['var_list']
    Efficiency =  x['cnt']
    Model_Name = x['region_name'][0]
    
    var_list = []
    for key in mydictionary:
        var_list.append((key, mydictionary[key],Efficiency,Model_Name))
          
    df = pandas.DataFrame(var_list, columns=['var_id', 'var_value','Efficiency','Model'])
    df_final = sqlContext.createDataFrame(df)
    
    return df_final 

## Generating Results from Throttles

In [ ]:
import pandas
region_df_algo_final = sqlContext.createDataFrame(region_df_algo_final) #changing back to sqlcontext

In [ ]:
field = [StructField("var_id", StringType(), True),StructField("var_value", DoubleType(), True),StructField("Efficiency", FloatType(), True),StructField("Model", StringType(), True)]
schema = StructType(field)
                                                                                                                      
optimization_df_algo1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)

for i in range(len(final_results)):
    optimization_df_algo1 = optimization_df_algo1.unionAll(df_create(final_results[i]))

optimization_df_algo1 = optimization_df_algo1.cache()

optimization_df_algo1 = optimization_df_algo1.withColumn("id",optimization_df_algo1["var_id"].cast(LongType()))
optimization_df_algo2 = region_df_algo_final.select([col for col in region_df_algo_final.columns if col not in {"predicted_new","net_revenue_new","objective_fun","Lower_Limit","var_name","part_id"}]).join(optimization_df_algo1,"id","inner")

In [ ]:
# here is where nathan is picking up on the code, going to include both training and validation sets
def threshold(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (1-z)

threshold_udf = udf(threshold,DoubleType())

def traffic_allow(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (z)

traffic_allow_udf = udf(traffic_allow,DoubleType())

In [ ]:
optimization_df_algo3 = optimization_df_algo2.withColumn("threshold",threshold_udf("var_value","AVERAGE_WEEKLY_TRAFFIC")).                        withColumn("traffic_allow",traffic_allow_udf("var_value","AVERAGE_WEEKLY_TRAFFIC"))

optimization_df_algo3 = optimization_df_algo3.select([col for col in optimization_df_algo3.columns if col not in {"id","var_id","Model"}])
optimization_df_algo3 = optimization_df_algo3.withColumnRenamed("var_value","Lp_solution")
optimization_df_algo_final = optimization_df_algo3.select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","threshold","EPB","traffic_allow")


In [ ]:
hcba_updatedcols = (hcba_opti_final.select('HOTEL_ID','LOS_BUCKET','BW_BUCKET','CPC_HCBA','AVERAGE_WEEKLY_TRAFFIC')
                                .withColumnRenamed('AVERAGE_WEEKLY_TRAFFIC','AVERAGE_WEEKLY_TRAFFIC-usedintraining')
                                .withColumnRenamed('CPC_HCBA','CPC_HCBA-usedintraining'))

In [ ]:
validation_df_spark = rf_validation_data
validation_df_spark1=validation_df_spark 

In [ ]:
def change_los(los):
    if los =="14-Aug":
        return "8-14"
    else :
        return los
change_los_udf = udf(change_los,StringType())

def change_bw(bw):
    if bw =="14-Aug":
        return "8-14"
    elif bw == "<=0":
        return "0"
    elif bw == "4-Mar":
        return "3-4"
    elif bw == "7-May":
        return "5-7"
    else :
        return bw
change_bw_udf = udf(change_bw,StringType())

In [ ]:
validation_df_spark2 = validation_df_spark1.withColumn("LOS_BUCKET_1",change_los_udf("LOS_BUCKET")).withColumn("BW_BUCKET_1",change_bw_udf("BW_BUCKET")).drop("LOS_BUCKET").drop("BW_BUCKET").withColumnRenamed("LOS_BUCKET_1","LOS_BUCKET").withColumnRenamed("BW_BUCKET_1","BW_BUCKET")

validation_df_spark3 = validation_df_spark2 


In [ ]:
common_queries = validation_df_spark3.join(optimization_df_algo_final.distinct(),["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

valid_df = common_queries.withColumn("throttle_traffic",(common_queries.traffic_allow*common_queries.TRAFFIC))

valid_df = valid_df.withColumn("throttle_cost",((valid_df.COST*valid_df.throttle_traffic)/valid_df.TRAFFIC))
valid_df = valid_df.withColumn("average_booking",(valid_df.BOOKING*valid_df.traffic_allow))
valid_df = valid_df.withColumn("average_earning",((valid_df.GROSS_PROFIT*valid_df.average_booking)/valid_df.BOOKING))

valid_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"}).show()

valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"}).show()

In [ ]:
validation_df_spark3.agg({"TRAFFIC":"sum","BOOKING":"sum","COST":"sum","GROSS_PROFIT":"sum"}).show()